In [2]:
import autogen
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent
from src.client import CustomModelClient

In [3]:
config_list_custom = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"model_client_cls": ["CustomModelClient"]},
)

In [2]:
# # Get the default system message
# default_system_message = AssistantAgent.DEFAULT_SYSTEM_MESSAGE

# # Custom additions
# custom_rules = """
# ADDITIONAL RULES:
# 1. Answer factual questions concisely.
# 2. Only suggest code when explicitly asked.
# 3. Always end with TERMINATE.
# """

# # Combine
# hybrid_message = f"{default_system_message}\n\n{custom_rules}"

In [8]:
assistant = AssistantAgent("assistant", 
                           llm_config={"config_list": config_list_custom})

[autogen.oai.client: 06-09 14:09:40] {890} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.


In [9]:
user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: (
        "TERMINATE" in x.get("content", "")),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  
    },
)

Register the custom client class to the assistant agent

In [10]:
assistant.register_model_client(model_client_cls=CustomModelClient)

Initialized Cerebras client for model: llama-4-scout-17b-16e-instruct


In [11]:
task = "How many minutes are there in an hour?"

# Start a conversation and get the final message only
chat_result = user_proxy.initiate_chat(
    assistant,
    message=task,
    summary_method="last_msg"  
)

if chat_result:
    print("\nFinal response:\n", chat_result.summary)

user_proxy (to assistant):

How many minutes are there in an hour?

--------------------------------------------------------------------------------
Incoming params: {'messages': [{'content': 'You are a helpful AI assistant.\nSolve tasks using your coding and language skills.\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\nSolve the task step by step if you need to. If a plan is not provided, explain